# Fase 1 del proyecto

En esta primera fase se limpiará los datos y se elegirá una problemática para brindar solución.

In [1]:
'''
Download necessary libraries if not have them already
'''
# !pip install numpy
# !pip install matplotlib.pyplot
# !pip install seaborn
# !pip install scikit-learn 
# !pip install pandas
# %pip install -r ../requirements.txt

'\nDownload necessary libraries if not have them already\n'

In [2]:
!python ../scripts/download_dataset.py

Cache deleted: C:\Users\josue/.cache/kagglehub/datasets/olistbr/brazilian-ecommerce
Extracting files...
File moved: olist_customers_dataset.csv
File moved: olist_geolocation_dataset.csv
File moved: olist_orders_dataset.csv
File moved: olist_order_items_dataset.csv
File moved: olist_order_payments_dataset.csv
File moved: olist_order_reviews_dataset.csv
File moved: olist_products_dataset.csv
File moved: olist_sellers_dataset.csv
File moved: product_category_name_translation.csv
Dataset successfully moved to: 'd:\UVG GitHub Repositorios\2025\CC3074_Proyecto_Final\data'



  0%|          | 0.00/42.6M [00:00<?, ?B/s]
  2%|▏         | 1.00M/42.6M [00:00<00:10, 4.16MB/s]
 12%|█▏        | 5.00M/42.6M [00:00<00:02, 17.7MB/s]
 28%|██▊       | 12.0M/42.6M [00:00<00:00, 33.2MB/s]
 42%|████▏     | 18.0M/42.6M [00:00<00:00, 41.9MB/s]
 63%|██████▎   | 27.0M/42.6M [00:00<00:00, 57.6MB/s]
 84%|████████▍ | 36.0M/42.6M [00:00<00:00, 67.5MB/s]
100%|██████████| 42.6M/42.6M [00:00<00:00, 51.3MB/s]


## Exploración de datos
Primero se va a chequear si existe datos nulos en los csv y recabar la cantidad de datos nullos encontrados por cada .csv del conjunto de datos. De igual forma
se verá qué tipos de datos se maneja y ver cantidatos para aplicar transformación a datos enteros. 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Callable, List, Tuple, Set
import os
import glob
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [4]:
# these functions will help with the data analysis 
# there's chance to add more information if deemeded necessary
def get_null_count(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Gives a count of nulls per column and its relative percentage.
    It does not change or replace nulls.
    params: 
        df: dataframe to get null info of
    returns:
        summary: dataframe that has a null count per col and a relative % of nulls in that col
    '''
    nulls: pd.Series = df.isnull().sum() # get the count of nulls per col
    percent: pd.Series = (nulls / len(df)) * 100 # get the percentage of nulls 
    summary: pd.DataFrame = pd.DataFrame({
        "Nulls" : nulls,
        "Percentage" : percent.round(2)
    })
    return summary[summary['Nulls'] > 0].sort_values(by='Nulls', ascending=False) # sort values by Nulls col , can change ascending to true

def get_types(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Gives the datatypes found within a dataframe.
    It does not change datatypes.
    params: 
        df: dataframe to get types of
    returns:
        types: dataframe with the type of each column
    '''
    types: pd.DataFrame = pd.DataFrame({
        'col_name' : df.columns,
        'data_type' : df.dtypes.astype(str).values
    })
    return types

def get_row_and_col_count(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Gives the number of rows and columns found in a dataframe
    params:
        df: dataframe to get num of cols and rows
    returns:
        num_row_col: dataframe with the number of columns and rows
    '''
    num_row_col: pd.DataFrame = pd.DataFrame([{
        'num_cols': len(df.columns),
        'num_rows': len(df)
    }])
    return num_row_col

def get_descriptive_stats(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Gives a brief description of the dataframe
    '''
    return df.describe().transpose()

def get_unique(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Return unique values found in the dataframe
    params:
        df: dataframe to get unique values from
    return:
        dataframe with unique values
    '''
    return df.nunique().to_frame(name='Unique Values')

def get_top_frequent(df: pd.DataFrame, top_n: int = 3) -> pd.DataFrame:
    '''
    Returns top 10 most frequent values found in the dataframe
    params: 
        df: dataframe to get unique values
        top_n: top number of values to get
    '''
    rows:list = []
    for col in df.select_dtypes(include='object').columns:
        top_vals = df[col].value_counts().head(top_n) # get the top 3 values
        for val, freq in top_vals.items(): # from those values, get the top value and its frequency
            rows.append({
                'Column Name': col,
                'Top Value': str(val), 
                'Frequency': freq
            })
    return pd.DataFrame(rows)

def write_report(df: pd.DataFrame, df_name: str, path: str, summary_functions: List[Callable]) -> None:
    '''
    Method that writes a report into a .txt of a certain dataframe
    params: 
        df: dataframe to be read
        path: where the .txt log will be saved
        summary_functions: a list functions that will be run and its returns be saved into the report
    returns:
        None
    '''
    if summary_functions is None:
        print("Please provide functions to be able to write a summary")
    
    with open(path, 'w') as log:
        log.write(f"DataFrame Summary Report Of {df_name}\n") # write header
        log.write("=" * 80 + "\n\n") 

        # for each function in the summary_functions list, write its summary and results
        for func in summary_functions:
            section_title: str = func.__name__.replace('_', ' ').title() # get the function name as title
            log.write(f"{section_title}\n")
            log.write("-" * len(section_title) + "\n")
            result_df: pd.DataFrame = func(df) # pass the dataframe argument to the function
            if result_df.empty:
                log.write("No information to report\n\n")
            else:
                log.write(result_df.to_string(index=True))
                log.write("\n\n")
        
        log.write("=" * 80 + "\nEnd\n") # finish the report


def generate_report_folder(csv_folder: str, report_folder: str, summary_functions: List[Callable]) -> None:
    '''
    Generates a report folder based upon the functions
    params:
        csv_folder: where the csv files are 
        report_folder: final folder where reports will be stored
        summary_functions: functions' returns that will be stored in the report
    '''
    os.makedirs(report_folder, exist_ok=True) # create folder if it does not exist
    csv_files: list = glob.glob(os.path.join(csv_folder, '*.csv'))
    for csv_path in csv_files:
        try:
            df: pd.DataFrame = pd.read_csv(csv_path)
            file_name: str = os.path.splitext(os.path.basename(csv_path))[0]
            report_path: str = os.path.join(report_folder, f"{file_name}_raw_report.txt")
            write_report(df=df, df_name=file_name, path=report_path, summary_functions=summary_functions)
            print(f"Report written into: {report_path}")
        except Exception as e:
            print(f"Failed processing {csv_path}: {e}")



In [5]:
# these reports are only based on each individual .csv, not on the database as a whole
generate_report_folder(csv_folder="../dataset_raw", report_folder="../dataset_reports_raw", summary_functions=[get_null_count, get_types, get_row_and_col_count, get_descriptive_stats, 
                                                                                                           get_top_frequent, get_unique])

## Transformación de datos
Al observar que hay ciertos conjuntos de datos con varios datos faltantes como en olist_order_reviews. Donde casi un 80% de las reseñas están vacías. Hay otras que también están vacías pero son cercanas a menos del 5%. Por lo que podemos decidir prescindir de estas sin problema alguno. Sin embargo, debemos considerar que esos porcentajes son locales en esos .csv, no son absolutos. Lo que quiere decir es que, no sabemos si los nulos de un csv son los mismos nulos de otro csv. Por ejemplo:

Si tenemos csv1 y csv2, asociados mediante un customer_id, y el csv1 tiene 4% de nulos y el csv2 tiene 2%, no sabemos si para el customer_id los nulos del csv1 y csv2 asociados a esa llave serán los mismos.
Por lo que tenemos que estar seguros que combinando la cantidad de nulos hallados en ambos, el total no sea mayor a 5%. Porque si los datos asociados al 4% de nulos son diferentes a los datos del 2% de nulos, estaríamos borrando un 6% de datos en general. 


Ahora bien, por lo mismo, al tener los datos normalizados en distintos .csv tenemos que considerar que si se elimina un dato de un .csv, se debe eliminar sus registros asociados en todos los .csv. De lo contrario, esto dará problemas a la hora de intentar modelar ya que el conjunto de datos estará incompleto o con datos faltantes en ciertos registros.

Entonces tenemos la elección de, o manejar los datos en el procesamiento de forma normalizada, o desnormalizar los datos y convegerlos en un solo conjunto de datos.

In [6]:
#TODO not been tested yet
def delete_nulls(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Deletes nulls from a dataframe if the count is less than 5%
    '''
    null: pd.DataFrame = get_null_count(df) 
    to_clean: list = null[null['Percentage'] > 5].index.tolist()
    if to_clean:
        return df.dropna(subset=to_clean)
    else:
        return df

#TODO possible approach to clean the dataset
def denormalize_dataset(dfs: List[pd.DataFrame]) -> pd.DataFrame:
    '''
    Denormalizes the datasets into one big dataset
    Input:
        dfs: a list of the dataframes to denormalize
    Return:
        a denormalized dataframe
    '''
    raise NotImplementedError()

### Limpieza e imputación

In [7]:
import pandas as pd
import numpy as np

orders = pd.read_csv("../data/olist_orders_dataset.csv", parse_dates=[
    "order_purchase_timestamp", "order_approved_at",
    "order_delivered_carrier_date", "order_delivered_customer_date",
    "order_estimated_delivery_date"
])

reviews = pd.read_csv("../data/olist_order_reviews_dataset.csv")
products = pd.read_csv("../data/olist_products_dataset.csv")
customers = pd.read_csv("../data/olist_customers_dataset.csv")
geolocation = pd.read_csv("../data/olist_geolocation_dataset.csv")

### Limpieza de nulos

In [8]:
# 1. Se eliminan ordenes sin fechas de entrega al cliente.
orders = orders[orders['order_delivered_customer_date'].notnull()]

# 2. Imputación de fechas utilizando la media
orders['order_delivered_carrier_date'] = orders['order_delivered_carrier_date'].fillna(
    orders['order_delivered_carrier_date'].median()
)
orders['order_approved_at'] = orders['order_approved_at'].fillna(
    orders['order_purchase_timestamp']
)

# 3. Flags para eomcentarios
reviews['has_title'] = reviews['review_comment_title'].notnull().astype(int)
reviews['has_message'] = reviews['review_comment_message'].notnull().astype(int)

# 4. Imputar nulos con mediana o texto para los productos
products['product_category_name'] = products['product_category_name'].fillna("unknown")
num_cols = [
    'product_name_lenght', 'product_description_lenght', 'product_photos_qty',
    'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm'
]
for col in num_cols:
    products[col] = products[col].fillna(products[col].median())

Se crea la variable target dificultad logística para poder clasificar el tretraso en tres niveles: fácil, moderada y difícil. Esto permite un mejor manejo de los datos.

In [9]:
orders['delivery_delay'] = (
    (orders['order_delivered_customer_date'] - orders['order_estimated_delivery_date']).dt.days
)
orders['dificultad_logistica'] = pd.cut(
    orders['delivery_delay'],
    bins=[-np.inf, 0, 5, np.inf],
    labels=["fácil", "moderada", "difícil"]
)
orders['delivery_time'] = (
    orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']
).dt.days

### Unificación de datasets

Se utilizacon las llaves para realizar los joins entre orders, reviews, order_items, etc. Esto con el objetivo de que cada fila del dataset contenga toda la información relevante para poder predecir la dificultad de la entrega.

In [10]:
# 1. Orders + Customers
df = orders.merge(customers, on='customer_id', how='left')

# 2. Orders + Reviews 
reviews_reduced = reviews[['order_id', 'review_score', 'has_title', 'has_message']]
df = df.merge(reviews_reduced, on='order_id', how='left')

# 3. Order Items 
order_items = pd.read_csv("../data/olist_order_items_dataset.csv")
df = df.merge(order_items[['order_id', 'product_id', 'freight_value', 'price']], on='order_id', how='left')

# 4. Products
df = df.merge(products, on='product_id', how='left')

#.5 Payments
payments = pd.read_csv("../data/olist_order_payments_dataset.csv")
payment_summary = payments.groupby("order_id").agg({
    "payment_type": lambda x: x.mode()[0] if not x.mode().empty else "unknown"
}).reset_index()
df = df.merge(payment_summary, on="order_id", how="left")

### Codificación

Se aplica One-Hot Encoding a variables categóricas como product_category_name y customer_state para convertir el texto a formato numérico. Las variables numéricas como freight_value y price se escalaron con StandarScaler para evitar la influencia de las diferencias de escala.

In [11]:
df['payment_type'] = df['payment_type'].fillna('unknown')  # si lo usás
df['product_category_name'] = df['product_category_name'].astype(str)

categorical_cols = ['customer_state', 'product_category_name']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

numerical_cols = ['freight_value', 'price', 'product_weight_g', 'delivery_time']
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

Separación entre variables predictorias y variable objetivo, eliminando nulos y columnas irrelevantes.

In [12]:
df = df[df['dificultad_logistica'].notnull()]

X = df.drop(columns=['dificultad_logistica', 'order_id', 'customer_id', 'product_id'])
y = df['dificultad_logistica']

df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_delay,dificultad_logistica,...,product_category_name_portateis_casa_forno_e_cafe,product_category_name_portateis_cozinha_e_preparadores_de_alimentos,product_category_name_relogios_presentes,product_category_name_seguros_e_servicos,product_category_name_sinalizacao_e_seguranca,product_category_name_tablets_impressao_imagem,product_category_name_telefonia,product_category_name_telefonia_fixa,product_category_name_unknown,product_category_name_utilidades_domesticas
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,-8,fácil,...,False,False,False,False,False,False,False,False,False,True
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,-6,fácil,...,False,False,False,False,False,False,False,False,False,False
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,-18,fácil,...,False,False,False,False,False,False,False,False,False,False
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,-13,fácil,...,False,False,False,False,False,False,False,False,False,False
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,-10,fácil,...,False,False,False,False,False,False,False,False,False,False
